### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
MS_T24_GF_310a = pd.read_csv("lca/cleaned/310_MS_T24_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
MS_T24_GF_310b = MS_T24_GF_310a[MS_T24_GF_310a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MS_T24_GF_310 = MS_T24_GF_310b[MS_T24_GF_310b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MS_T24_GF_310))

MS_T24_GF_310.head(6)

# of phylum peptides =  24


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
1,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,MHLMHLTK,Dorea,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Dorea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,DVEEMVLR,Chloroflexi,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,QEFLNAAK,Phorcysia thermohydrogeniphila,Bacteria,NaN,NaN,NaN,Aquificae,NaN,NaN,Aquificae,...,Phorcysia,NaN,NaN,NaN,Phorcysia thermohydrogeniphila,NaN,NaN,NaN,NaN,NaN
91,VEVVSLPR,Actinomycetia,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [5]:
MS_T24_GF_310_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/310A_MS_T24_GF_DN50_ILnaafs.csv")
MS_T24_GF_310_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/310B_MS_T24_GF_DN50_ILnaafs.csv")
MS_T24_GF_310_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/310C_MS_T24_GF_DN50_ILnaafs.csv")

frames = [MS_T24_GF_310_NAAFa, MS_T24_GF_310_NAAFb, MS_T24_GF_310_NAAFc]

MS_T24_GF_310_NAAF = pd.concat(frames, sort=False)

MS_T24_GF_310_NAAF.set_index('stripped_peptide')
MS_T24_GF_310_NAAF = MS_T24_GF_310_NAAF.loc[:, ~MS_T24_GF_310_NAAF.columns.str.contains('^Unnamed')]

MS_T24_GF_310_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MS_T24_GF_310_NAAF))

print('column names:', MS_T24_GF_310_NAAF.columns)

MS_T24_GF_310_NAAF.head()

# of total peptides =  494
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,LSSPATLNSR,35200000.0,3520000.0
1,LSSPATLNSR,35200000.0,3520000.0
2,LSSPATLNSR,35200000.0,3520000.0
3,LATVLSPR,14900000.0,1862500.0
4,LATVLSPR,14900000.0,1862500.0


In [6]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MS_T24_GF_310["peptide"], MS_T24_GF_310_NAAF["peptide"])

print(over)

['DVEEMVLR' 'EFHPGLAR' 'LATVLSPR' 'QEFLNAAK' 'SPALFWPSK' 'VATVSPLR'
 'VEVVSLPR' 'VVMVLPPK']


In [7]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MS_T24_GF_310.set_index('peptide', inplace=True)
MS_T24_GF_310_NAAF.set_index('peptide', inplace=True)

MS_T24_GF_310_Phy = MS_T24_GF_310.join(MS_T24_GF_310_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MS_T24_GF_310_Phy))

MS_T24_GF_310_Phy.head()

# of total phylum-level peptides =  52


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
AGLVNSLALK,Actinomycetia,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ATNPTLLK,Ruminococcus bromii,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,NaN,...,NaN,NaN,Ruminococcus bromii,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DVEEMVLR,Chloroflexi,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19200.0,2400.0
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,1040000.0,130000.0
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,5960000.0,745000.0


In [8]:
# write to a csv

MS_T24_GF_310_Phy.to_csv("lca/NAAF/MS_T24_GF_310_Phy_naaf.csv")